In [1]:
import pandas as pd
import numpy as np

# Adjust notebook settings to widen the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:80% !important;}</style>"))
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
hpsa = pd.read_csv("./Data/HPSA_Cleaned.csv")
nsduh = pd.read_csv("./Data/nsduh_data_cleaned.csv")
grants = pd.read_csv("./Data/grants_per_county_cbsa.csv")

In [3]:
print('HPSA shape:{}'.format(hpsa.shape))
print('Grants shape:{}'.format(grants.shape))
print('NSDUH shape:{}'.format(nsduh.shape))

# HPSA shape:(27829, 52)
# Grants shape:(2329, 11)
# NSDUH shape:(214505, 96)

HPSA shape:(27829, 52)
Grants shape:(2329, 11)
NSDUH shape:(214505, 96)


In [4]:
grants = grants.rename(columns={'Award Year':'Year', 'Complete County Name':'County Name'})
grants.drop(['centraloutlyingcounty', 'countycountyequivalent', 'statename', 'cbsacode', 'metropolitanmicropolitanstatis'], axis=1, inplace=True)

In [5]:
grants.head(15)

,County Name,State Name,Year,Total Active Grant Financial Assistance,Mental Health Assistance,PDEN10
0,Accomack County,Virginia,2017,52817.0,0.0,3
1,Accomack County,Virginia,2018,4973949.0,0.0,3
2,Accomack County,Virginia,2019,5397224.0,0.0,3
3,Ada County,Idaho,2017,1560323.0,10000.0,1
4,Ada County,Idaho,2018,3254149.0,225000.0,1
5,Ada County,Idaho,2019,4940351.0,225000.0,1
6,Adair County,Missouri,2018,159703.0,0.0,2
7,Adair County,Missouri,2019,3204629.0,0.0,2
8,Adams County,Colorado,2016,21576.0,0.0,1
9,Adams County,Colorado,2017,310347.0,0.0,1


### HPSA

In [6]:
# drop unrecognized hpsas
hpsa = hpsa[~hpsa['metropolitanmicropolitanstatis'].isnull()]

In [7]:
hpsa['HPSA Withdrawn Year'] = pd.to_datetime(hpsa['Withdrawn Date']).dt.year

In [8]:
hpsa['HPSA Designation Year'] = pd.to_datetime(hpsa['HPSA Designation Date']).dt.year

In [9]:
# remove repetitive columns & not useful columns for Tableau visualizations
hpsa.drop(['Common State County FIPS Code', 
         'Common State FIPS Code',
         'Common State Name', 
         'Common State Abbreviation',
         'County Equivalent Name',
         'HPSA Metropolitan Indicator Code', 
         'Primary State FIPS Code', 
         'State Abbreviation', 
         'State FIPS Code', 
         'State Name', 
         'Common County Name', 
         'Metropolitan Indicator',
         'HPSA Status Code',  
         'Rural Status Code',  
         'HPSA Component State Abbreviation',
         'HPSA Population Type Code', 
         'countycountyequivalent',
         'statename',
         'cbsacode',
         'metropolitanmicropolitanstatis',
         'centraloutlyingcounty'], axis=1, inplace=True)

In [10]:
hpsa.head(2)

,Withdrawn Date,HPSA Name,HPSA Component Name,HPSA Component Type Code,HPSA Component Type Description,HPSA Designation Population Type Description,HPSA Type Code,Primary State Name,State and County Federal Information Processing Standard Code,U.S. - Mexico Border 100 Kilometer Indicator,U.S. - Mexico Border County Indicator,Common Region Name,County or County Equivalent Federal Information Processing Standard Code,HPSA Designation Last Update Date,HPSA Designation Date,HPSA Status,HPSA Geography Identification Number,HPSA Score,Primary State Abbreviation,Primary HHS Region Name,Designation Type,HPSA ID,HPSA Designation Population,Rural Status,HPSA Degree of Shortage,HPSA FTE,HPSA Population Type,HPSA Shortage,DaysBeforeWithdrawn,County Equivalent Name New,PDEN10,HPSA Withdrawn Year,HPSA Designation Year
0,NaN,Stanley Correctional Institution,Stanley Correctional Institution,UNK,Unknown,Correctional Facility,PRSN,Wisconsin,55017,N,N,Region 5,017,08/02/2018,07/21/2003,Designated,POINT,15,WI,Region 5,Correctional Facility,7551065910,2885.0,Non-Rural,6,0.6,NaN,0.84,NaN,Chippewa County,1,NaN,2003
1,07/02/2018,Rock County,Rock,SCTY,Single County,Geographic Population,Hpsa Geo HN,Wisconsin,55105,N,N,Region 5,105,07/02/2018,04/09/2014,Withdrawn,55105,13,WI,Region 5,High Needs Geographic HPSA,755105,156639.0,Partially Rural,Not applicable,8.5,Geographic Population,0.74,1545 days,Rock County,1,2018.0,2014


## NSDUH

In [11]:
nsduh.drop(['Unnamed: 0'], axis=1, inplace=True)

## Merge all datasets (NSDUH, HPSA, Grants)

In [12]:
# merge HPSA and Grant by Year, State, County, and PDEN10
hpsa_grants = hpsa.merge(grants, how='left', left_on=['PDEN10', 'HPSA Designation Year', 'Primary State Name', 'County Equivalent Name New' ], right_on=['PDEN10', 'Year', 'State Name', 'County Name'])

In [13]:
# show only HPSA Designation Year >=2015 and <2020
hpsa_grants = hpsa_grants.loc[(hpsa_grants['HPSA Designation Year']>=2015) & (hpsa_grants['HPSA Designation Year']<=2019)]

In [14]:
hpsa_grants = hpsa_grants[~hpsa_grants['Total Active Grant Financial Assistance'].isnull()]

In [15]:
hpsa_grants[['HPSA Designation Date','Withdrawn Date']] = hpsa_grants[['HPSA Designation Date','Withdrawn Date']].apply(pd.to_datetime) #if conversion required
hpsa_grants['DaysBeforeWithdrawn'] = (hpsa_grants['Withdrawn Date'] - hpsa_grants['HPSA Designation Date']).dt.days

In [16]:
hpsa_grants.shape # (4783, 38)
hpsa_grants.head(2)

,Withdrawn Date,HPSA Name,HPSA Component Name,HPSA Component Type Code,HPSA Component Type Description,HPSA Designation Population Type Description,HPSA Type Code,Primary State Name,State and County Federal Information Processing Standard Code,U.S. - Mexico Border 100 Kilometer Indicator,U.S. - Mexico Border County Indicator,Common Region Name,County or County Equivalent Federal Information Processing Standard Code,HPSA Designation Last Update Date,HPSA Designation Date,HPSA Status,HPSA Geography Identification Number,HPSA Score,Primary State Abbreviation,Primary HHS Region Name,Designation Type,HPSA ID,HPSA Designation Population,Rural Status,HPSA Degree of Shortage,HPSA FTE,HPSA Population Type,HPSA Shortage,DaysBeforeWithdrawn,County Equivalent Name New,PDEN10,HPSA Withdrawn Year,HPSA Designation Year,County Name,State Name,Year,Total Active Grant Financial Assistance,Mental Health Assistance
147,NaT,ICE - El Paso Processing Center,ICE - El Paso Processing Center,UNK,Unknown,Correctional Facility,PRSN,Texas,48141,Y,Y,Region 6,141,09/05/2019,2019-05-13,Designated,POINT,21,TX,Region 6,Correctional Facility,7489517683,710.0,Non-Rural,12,0.0,NaN,0.36,NaN,El Paso County,1,NaN,2019,El Paso County,Texas,2019.0,12142926.0,0.0
164,2019-06-25,CF-Joe Kegans State Jail,CF-Joe Kegans State Jail,UNK,Unknown,Correctional Facility,PRSN,Texas,48201,N,N,Region 6,201,06/25/2019,2016-07-08,Withdrawn,POINT,6,TX,Region 6,Correctional Facility,7482828719,NaN,Non-Rural,6,0.0,NaN,0.25,1082.0,Harris County,1,2019.0,2016,Harris County,Texas,2016.0,316515.0,0.0


In [17]:
# import census region dataset
# retrieved from: https://github.com/cphalpert/census-regions 
census_region = pd.read_csv('./Data/census_region.csv')

In [18]:
census_region.drop(['State Code'], axis=1, inplace=True)
census_region.head(2)

,State,Region,Division
0,Alaska,West,Pacific
1,Alabama,South,East South Central


In [19]:
# add region & division to hpsa_grants dataset
hpsa_grants_region = hpsa_grants.merge(census_region, how='left', left_on=['Primary State Name' ], right_on=['State'])
hpsa_grants_region.head(2)

,Withdrawn Date,HPSA Name,HPSA Component Name,HPSA Component Type Code,HPSA Component Type Description,HPSA Designation Population Type Description,HPSA Type Code,Primary State Name,State and County Federal Information Processing Standard Code,U.S. - Mexico Border 100 Kilometer Indicator,U.S. - Mexico Border County Indicator,Common Region Name,County or County Equivalent Federal Information Processing Standard Code,HPSA Designation Last Update Date,HPSA Designation Date,HPSA Status,HPSA Geography Identification Number,HPSA Score,Primary State Abbreviation,Primary HHS Region Name,Designation Type,HPSA ID,HPSA Designation Population,Rural Status,HPSA Degree of Shortage,HPSA FTE,HPSA Population Type,HPSA Shortage,DaysBeforeWithdrawn,County Equivalent Name New,PDEN10,HPSA Withdrawn Year,HPSA Designation Year,County Name,State Name,Year,Total Active Grant Financial Assistance,Mental Health Assistance,State,Region,Division
0,NaT,ICE - El Paso Processing Center,ICE - El Paso Processing Center,UNK,Unknown,Correctional Facility,PRSN,Texas,48141,Y,Y,Region 6,141,09/05/2019,2019-05-13,Designated,POINT,21,TX,Region 6,Correctional Facility,7489517683,710.0,Non-Rural,12,0.0,NaN,0.36,NaN,El Paso County,1,NaN,2019,El Paso County,Texas,2019.0,12142926.0,0.0,Texas,South,West South Central
1,2019-06-25,CF-Joe Kegans State Jail,CF-Joe Kegans State Jail,UNK,Unknown,Correctional Facility,PRSN,Texas,48201,N,N,Region 6,201,06/25/2019,2016-07-08,Withdrawn,POINT,6,TX,Region 6,Correctional Facility,7482828719,NaN,Non-Rural,6,0.0,NaN,0.25,1082.0,Harris County,1,2019.0,2016,Harris County,Texas,2016.0,316515.0,0.0,Texas,South,West South Central


In [20]:
# save as hpsa_grants.cleaned.csv
hpsa_grants.to_csv('./Data/hpsa_grants_data_cleaned.csv',index=False)
hpsa_grants_region.to_csv('./Data/hpsa_grants_region_data_cleaned.csv',index=False)

In [24]:
# 25th Percentile
def q25(x):
    return x.quantile(0.25)

# 75th Percentile
def q75(x):
    return x.quantile(0.75)

hpsa_grants_agg = hpsa_grants_region.groupby(['PDEN10', 'Year'], as_index=False).agg(
    {'Total Active Grant Financial Assistance':['mean', 'median', q25, q75],
     'Mental Health Assistance':'mean',
     'DaysBeforeWithdrawn':['mean', 'min', 'max']})

# # flatten hierarchical index 
hpsa_grants_agg.columns = [' '.join(col).strip() for col in hpsa_grants_agg.columns.values]
hpsa_grants_agg = hpsa_grants_agg.reset_index()

In [25]:
# merge with nsduh dataset 
final_df = nsduh.merge(hpsa_grants_agg, how='left', left_on=['PDEN10', 'Year' ], right_on=['PDEN10', 'Year'])

In [27]:
final_df.head(2)

,Id,Year,Inpatient_Past_Year,Outpatient_Past_Year,Prescription_Treatment_Past_Year,Any_Treatment_Past_Year,Treatment_Type_Past_Year,Perceived_Unmet_Need,Received_Treatment_At_Clinic_Or_Center,Received_Treatment_At_Private_Therapist,Received_Treatment_At_NonClinic_Doctor,Received_Treatment_At_Medical_Clinic,Received_Treatment_At_Day_Hospital,Received_Treatment_At_School,Received_Treatment_Other,Self_Paid,Non_Household_Member_Paid,Private_Health_Insurance_Paid,Medicare_Paid,Medicaid_Paid,Rehab_Paid,Employer_Paid,Military_Paid,Other_Public_Source_Paid,Other_Private_Source_Paid,Nobody_Paid,No_Treatment_Could_Not_Afford,No_Treatment_Feared_Neighbors_Opinion,No_Treatment_Feared_Effect_On_Job,No_Treatment_Insurance_Not_Covered,No_Treatment_Insurance_Not_Covered_Enough,No_Treatment_Where_To_Go,No_Treatment_Confidentiality_Concerns,No_Treatment_Fear_Of_Being_Committed,No_Treatment_Didnt_Think_Needed,No_Treatment_Handle_Problem_Without,No_Treatment_Didnt_Think_Would_Help,No_Treatment_Didnt_Have_Time,No_Treatment_Didnt_Want_Others_To_Know,No_Treatment_No_Transport_Inconvenient,No_Treatment_Other,Num_Weeks_Mental_Health_Difficulties,Num_Days_Past_Year_Unable_To_Work,Serious_Psychological_Distress_Indicator_Past_Month,Psychological_Distress_Level_Worst_Month,Worst_Psychological_Distress_Level,Serious_Psychological_Distress_Indicator_Past_Year,Serious_Suicidal_Thoughts_Past_Year,Suicide_Plan_Past_Year,Suicide_Attempt_Past_Year,Serious_Or_Moderate_Mental_Illness_Indicator_Past_Year,Moderate_Mental_Illness_Indicator_Past_Year,Mild_Mental_Illness_Indicator_Past_Year,Low_Or_Moderate_Mental_Illness_Indicator_Past_Year,Categorical_Mental_Illness_Indicator,Serious_Mental_Illness_And_Substance_Abuse,Any_Mental_Illness_And_Substance_Abuse,Low_Or_Moderate_Mental_Illness_And_Substance_Abuse,Adult_Lifetime_Major_Depressive_Episode,Adult_Past_Year_Major_Depressive_Episode,Adult_Received_Counseling_Or_Meds_For_Depressive_Feelings_Past_Year,Gender,Age_Category,Age_Category_Two_Levels,Age_Category_Three_Levels,Age_Category_Six_Levels,Gender_Age_Category,Race_Ethnicity,Race_Sex,Education_Category,Overall_Health,Work_Situation_Past_Week,Num_Days_Skipped_Work_Past_30_Days,EAP_Offered,Adult_Employment_Status,Has_Medicare,Has_Medicaid_Or_CHIP,Has_Military_Benefit,Has_Private_Health_Insurance,Has_Other_Health_Insurance,Covered_By_Any_Health_Insurance,Covered_By_Any_Health_Insurance_Imputation_Revised,Family_Receives_Social_Security,Family_Receives_SSI,Family_Receives_Food_Stamps,Family_Receives_Public_Assistance,Family_Receives_Welfare_JobPlacement_Childcare,Months_On_Welfare,Total_Income_Respondent,Total_Income_Family,Participated_In_One_Or_More_Government_Assistance_Programs,Total_Income_Family_Recode,Poverty_Level,PDEN10,County_Metro_NonMetro_Status,index,Total Active Grant Financial Assistance mean,Total Active Grant Financial Assistance median,Total Active Grant Financial Assistance q25,Total Active Grant Financial Assistance q75,Mental Health Assistance mean,DaysBeforeWithdrawn mean,DaysBeforeWithdrawn min,DaysBeforeWithdrawn max
0,13005143,2015,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,4,3,4,4,5,7,5,2,1,1,0,0,1,0,1,0,0,-1,1,1,0,0,1,0,1,1,2,4,1,2,2,2,2,5.0,21111.0,24533.0,19400.0,24533.0,0.0,1855.0,1855.0,1855.0
1,67415143,2015,0,1,1,1,6,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,0,0,11,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,2,3,3,3,3,5,1,2,4,2,7,0,-1,4,0,0,0,1,-1,1,1,0,0,0,0,0,0,1,1,0,1,1,2,3,5.0,21111.0,24533.0,19400.0,24533.0,0.0,1855.0,1855.0,1855.0


In [28]:
final_df.drop(['index'], axis=1, inplace=True)
final_df.shape # (214505, 103)

(214505, 103)

In [29]:
## save merged HPSA, NSDUH, Grant to ./Data folder 
final_df.to_csv('./Data/nsduh_hpsa_grants_cleaned_data_for_Tableau.csv',index=False)